In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

## EDA + 1st submit

In [2]:
import matplotlib.pyplot as plt
from lightgbm import LGBMRegressor
from sklearn.preprocessing import StandardScaler, OrdinalEncoder



In [3]:
data = pd.read_csv('/kaggle/input/jpx-tokyo-stock-exchange-prediction/train_files/stock_prices.csv')
# display(data)

In [4]:
# hold-out validation  2021 전후로 나눔
# train = data[data['Date'] < '2021-01-01']
# val = data[data['Date'] >= '2021-01-01']
train = data.copy()
# print(train.head(3), val.head(3))

In [5]:
# len(train), len(val)

In [6]:
def add_features(dfc):
    df = dfc.copy()
    # shadows
    df['upper_shadow'] = df['High'] - np.maximum(df['Open'], df['Close'])
    df['lower_shadow'] = np.minimum(df['Open'], df['Close']) - df['Low']
    
    # lagged features
    # 날짜 단위이므로 7일전, 30일전, 180일전, 360일전 
    # lagged close, target (target 은 정확히 무엇? return인가)
    
    # lagged ema
    df['Date'] = pd.to_datetime(df['Date'])
    return df

co_train = add_features(train)
# co_train

In [7]:
# 시각화
# def code_plot(number, y='Close'):
#     ex1 = co_train[co_train.SecuritiesCode == number]
#     plt.plot(ex1.Date, ex1[y])
#     plt.xlabel('Date')
#     plt.ylabel(y)
    
    
# code_plot(1301,'Target')

'''
base model - lgbm (maybe ensemble with nn?)
features - close + lagged features + window

feature 추가할것
1. Lagged features
2. lagged ema
3. shadows

요일, 월 정도 추가 
ACF 적용해서 lagged 어느정도 적용하는게 좋은지 판단하기.

features should be normalized - StandardScaler
'''

In [8]:
def preprocess(df, train=True):
    dfc = df.copy()
    
    # remove columns - Date removed temporarily
    dfc = dfc.drop(columns=['Date', 'RowId', 'AdjustmentFactor', 'ExpectedDividend', 'SupervisionFlag'])
    stdsc = StandardScaler()
    ordinal = OrdinalEncoder()

    target = ['Target']
    ord_features = ['SecuritiesCode']
    scaled_features = ['Open', 'High', 'Low', 'Close', 'Volume', 'upper_shadow', 'lower_shadow']

    date_code_ord = ordinal.fit_transform(dfc.loc[:,ord_features])
    scaled = stdsc.fit_transform(dfc.loc[:,scaled_features])
    dfc_scaled = pd.concat([pd.DataFrame(date_code_ord, columns=ord_features),
                            pd.DataFrame(scaled, columns=scaled_features)], axis=1)
    
    if train:
        y = dfc.loc[:,target]
        return dfc_scaled, y
    else:
        return dfc_scaled

X_scaled, y = preprocess(co_train)

# X_scaled, y

In [9]:
# base model - lgbm 
lgb = LGBMRegressor().fit(X_scaled, y)


In [10]:
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

In [11]:
for (prices, options, financials, trades, secondary_prices, sample_prediction) in iter_test:
    feat = add_features(prices)
    X = preprocess(feat, train=False)

    # X, y
    X['Target'] = lgb.predict(X)
    X['Rank'] = (X['Target'].rank(method='first', ascending=False)-1).astype(int)
    sample_prediction['Rank'] = X['Rank'].values
#     display(sample_prediction)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.
